# Getting Started with PnP Core SDK - Application permissions

This solution demonstrates how the PnP Core SDK can be used in a Polyglot notebook while using application permissions using a certificate for authentication. In this sample we're showing you how you can use PnP Core SDK using the minimal amount of configuration and code.

This notebook is a demonstration based from the minimal console sample to running fragments of code in a Polyglot notebook, that contains WORKING code and returns results base don a secure connection to your tenant

## Prerequisites

### Getting started

Before this Polyglot notebook can run you need to:
- Install the latest [Visual Studio Code](https://code.visualstudio.com/).
- Install the latest [.NET 6 SDK](https://dotnet.microsoft.com/download/dotnet/6.0).
- Install the Polyglot Notebooks extension from the [marketplace](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode).

### Setup an Azure AD app for accessing SharePoint

The one thing to configure before you can use this sample is an Azure AD application:

1. Navigate to [Azure AD Portal](https://aad.portal.azure.com/) and click on **Azure Active Directory** from the left navigation
2. Click on **Add** in the toolbar and choose **App registration**, this will open up the **Register an application** page
3. Provide a **Name** for your Azure AD application (e.g. Microsoft365AssessmentToolForWorkflow)
4. Select **Public client/native (mobile & desktop)** and enter **http://localhost** as redirect URI
5. Click on **Register** and the Azure AD application gets created and opened
6. Click on **API permissions** and add these **application** permissions
   1. Microsoft Graph -> Sites.Read.All
   2. SharePoint -> Sites.Read.All
7. Click on **Grant admin consent for...** to consent the added permissions
8. Go to **Certificates & secrets**, click on **Certificates** and **Upload certificate**, pick the .cer file of your certificate and add it.

Note: If you don't have a certificate available then you can [use Windows PowerShell to create one](https://docs.microsoft.com/en-us/azure/active-directory/develop/howto-create-self-signed-certificate).

### Setting up a connection in the notebook and reference the libraries

Firstly, for the notebook to run, you need to get libraries from nuget and setup the using statements with the following call:

In [19]:
// Only install PnP.Core.Auth, this will pull in PnP.Core
#r "nuget: PnP.Core.Auth,*-*"
#r "nuget: Microsoft.Extensions.Hosting, 6.0.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;
using PnP.Core.Auth.Services.Builder.Configuration;
using PnP.Core.Services;
using PnP.Core.Services.Builder.Configuration;
using System.Security.Cryptography.X509Certificates;

Installed Packages Microsoft.Extensions.Hosting, 6.0.0 PnP.Core.Auth, 1.8.11-nightly

### Set the connection variables

Replace each of these following variables with values from your Azure AD application and you're own SharePoint site. 

In [20]:
// Replace the client ID with the Application ID from the Azure AD app registration
string clientId = "c545f9ce-1c11-440b-812b-0b35217d9e83";

// Specify your Azure AD tenant id
string tenantId = "d8623c9e-30c7-473a-83bc-d907df44a26e";

// Specify the thumbprint of the certificate to use, the certificate must be present in the LocalUser/My location
string certificatePath = "b133d1cb4d19ce539986c7ac67de005481084c84";

// Set the URL to the SharePoint site you want to connect to
string siteUrl = "https://bertonline.sharepoint.com/sites/prov-1";

Configure the host using the `PnPCoreAuthenticationX509CertificateOptions` authentication provider. 

In [21]:
// Creates and configures the host
var host = Host.CreateDefaultBuilder()
    .ConfigureServices((context, services) => 
    {
        // Add PnP Core SDK
        services.AddPnPCore(options =>
        {
            // Add the base site url
            options.Sites.Add("Default", new PnPCoreSiteOptions
            {
                SiteUrl = siteUrl
            });
        });

        // Manual configure the used authentication
        services.AddPnPCoreAuthentication(options =>
        {
            // Load the certificate that will be used to authenticate
            //var cert = LoadCertificate(certificatePath);

            // Configure certificate based auth
            options.Credentials.Configurations.Add("CertAuth", new PnPCoreAuthenticationCredentialConfigurationOptions
            {
                ClientId = clientId,
                TenantId = tenantId,
                X509Certificate = new PnPCoreAuthenticationX509CertificateOptions
                {
                    Certificate = LoadCertificate(certificatePath),
                }
            });

            // Configure the default authentication provider
            options.Credentials.DefaultConfiguration = "CertAuth";
            
            // Connect the configured authentication method to the configured site
            options.Sites.Add("Default", new PnPCoreAuthenticationSiteOptions
            {
                AuthenticationProviderName = "CertAuth",
            });

            options.Credentials.DefaultConfiguration = "CertAuth";
        });

    })
    .UseConsoleLifetime()
    .Build();

// Start the host
await host.StartAsync();

private static X509Certificate2 LoadCertificate(string certificateThumbprint)
{
    var store = new X509Store(StoreName.My, StoreLocation.CurrentUser);
    store.Open(OpenFlags.ReadOnly | OpenFlags.OpenExistingOnly);
    var certificateCollection = store.Certificates.Find(X509FindType.FindByThumbprint, certificateThumbprint, false);
    store.Close();

    return certificateCollection.First();
}

info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: d:\github\pnpcore\docs\polyglot


# The application

Now that we have setup the prerequisites, we can start using the PnP Core SDK in the notebook.

In [ ]:
using (var scope = host.Services.CreateScope())
{
    // Ask an IPnPContextFactory from the host
    var pnpContextFactory = scope.ServiceProvider.GetRequiredService<IPnPContextFactory>();

    // Create a PnPContext
    using (var context = await pnpContextFactory.CreateAsync(new Uri(siteUrl)))
    {
        // Load the Title property of the site's root web
        await context.Web.LoadAsync(p => p.Title);
        Console.WriteLine($"The title of the web is {context.Web.Title}");
    }
}